<a href="https://colab.research.google.com/github/hammadnajeeb123/EcodeCamp_Internships/blob/main/task_no_2_twitter_sentiment_analysis_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir -p~/.kaggle/
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

mkdir: invalid option -- '~'
Try 'mkdir --help' for more information.
cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [2]:
#api to fetch the data set from kaggle
!kaggle datasets download -d kazanova/sentiment140
#shift + enter to dowanload

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 99% 80.0M/80.9M [00:03<00:00, 32.0MB/s]
100% 80.9M/80.9M [00:03<00:00, 24.8MB/s]


In [3]:
#dowanloaded in zip file extract from zip file
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'    #path copied of dowanloaded file
with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('the data set is extracted ! ')

the data set is extracted ! 


In [4]:
#importing the dependinces
import pandas as pd
import numpy as np
import re # regular expressions in order to do patrenn matching
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score #to calculate the accuracy score

In [5]:
import nltk
nltk.download('stopwords')
#

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
#printing stopwords in english
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

data processing


In [7]:
#loading the data from csv file to pandas data frame
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='iso-8859-1')

In [8]:
#checking the number of rows and colunms
twitter_data.shape

(1599999, 6)

In [9]:
#printing first five rows of the data set
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [10]:

#first value is been read as column name no we have to tell the model
#naming the column and reading the data set again
column_names = ['target','ids','date','flag','user','text']
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='iso-8859-1',names=column_names)
twitter_data.shape

(1600000, 6)

In [11]:
#now reading again
twitter_data.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [12]:
#checking missing values
#countion the number of missing values or data sets
twitter_data.isnull().sum()

,0
target,0
ids,0
date,0
flag,0
user,0
text,0


NO missing data set or values

distributing the target column to check how many are positive or negative

In [13]:
twitter_data['target'].value_counts()

,count
target,
0,800000
4,800000


In [14]:
#changing te lable 0 and 4 to 0 and 1
twitter_data.replace({'target':{0:0,4:1}},inplace=True)  #inplace is to store the data
twitter_data['target'].value_counts()

,count
target,
0,800000
1,800000


0-->means negative twitte
1-->means positive twitte

**stemming**

stemming is the process to reduce a word to its root word

In [15]:
#example actor ,actress ,acting = act
from nltk.stem import PorterStemmer # Import the PorterStemmer class

port_stem = PorterStemmer()

In [16]:
import re
from nltk.corpus import stopwords

def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content) # remove non-alphabetical characters
  stemmed_content = stemmed_content.lower().split() # convert to lowercase and split into words
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')] # apply stemming and remove stop words
  stemmed_content = ' '.join(stemmed_content) # join the stemmed words back into a string
  return stemmed_content
  #ml model can not use the phrases or sentences so we do stemming
  #line 1 removes @ , and numbers like a2 23l etc etc
  #line 2 split the words in the list stored


it will take 15 to 60 min to complete due to 16M comments

In [17]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [18]:
twitter_data.head()

,target,ids,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [19]:
print(twitter_data['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [20]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


separating the data and lable

In [21]:
x=twitter_data['stemmed_content'].values
y=twitter_data['target'].values

In [22]:
#all comments 16M
print(x)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [23]:
#1st three and last 3 targets
print(y)

[0 0 0 ... 1 1 1]


spliting the data to train and test data

In [24]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,stratify=y,random_state=2)


In [25]:
print(x.shape,x_train.shape,x_test.shape)

(1600000,) (1280000,) (320000,)


In [26]:
print(x_train)

['watch saw iv drink lil wine' 'hatermagazin'
 'even though favourit drink think vodka coke wipe mind time think im gonna find new drink'
 ... 'eager monday afternoon'
 'hope everyon mother great day wait hear guy store tomorrow'
 'love wake folger bad voic deeper']


In [27]:
print(y_train)

[1 1 0 ... 1 1 0]


In [28]:
print(x_test)

['mmangen fine much time chat twitter hubbi back summer amp tend domin free time'
 'ah may show w ruth kim amp geoffrey sanhueza'
 'ishatara mayb bay area thang dammit' ...
 'destini nevertheless hooray member wonder safe trip' 'feel well'
 'supersandro thank']


In [29]:
print(y_test)

[0 1 0 ... 1 0 1]


ML model cannot read variable now converting into numerical data

In [30]:
vectorizer = TfidfVectorizer()
x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)

In [31]:
print(x_train)

  (0, 443066)	0.4484755317023172
  (0, 235045)	0.41996827700291095
  (0, 109306)	0.3753708587402299
  (0, 185193)	0.5277679060576009
  (0, 354543)	0.3588091611460021
  (0, 436713)	0.27259876264838384
  (1, 160636)	1.0
  (2, 288470)	0.16786949597862733
  (2, 132311)	0.2028971570399794
  (2, 150715)	0.18803850583207948
  (2, 178061)	0.1619010109445149
  (2, 409143)	0.15169282335109835
  (2, 266729)	0.24123230668976975
  (2, 443430)	0.3348599670252845
  (2, 77929)	0.31284080750346344
  (2, 433560)	0.3296595898028565
  (2, 406399)	0.32105459490875526
  (2, 129411)	0.29074192727957143
  (2, 407301)	0.18709338684973031
  (2, 124484)	0.1892155960801415
  (2, 109306)	0.4591176413728317
  (3, 172421)	0.37464146922154384
  (3, 411528)	0.27089772444087873
  (3, 388626)	0.3940776331458846
  (3, 56476)	0.5200465453608686
  :	:
  (1279996, 390130)	0.22064742191076112
  (1279996, 434014)	0.2718945052332447
  (1279996, 318303)	0.21254698865277746
  (1279996, 237899)	0.2236567560099234
  (1279996, 2910

In [32]:
print(x_test)

  (0, 420984)	0.17915624523539803
  (0, 409143)	0.31430470598079707
  (0, 398906)	0.3491043873264267
  (0, 388348)	0.21985076072061738
  (0, 279082)	0.1782518010910344
  (0, 271016)	0.4535662391658828
  (0, 171378)	0.2805816206356073
  (0, 138164)	0.23688292264071403
  (0, 132364)	0.25525488955578596
  (0, 106069)	0.3655545001090455
  (0, 67828)	0.26800375270827315
  (0, 31168)	0.16247724180521766
  (0, 15110)	0.1719352837797837
  (1, 366203)	0.24595562404108307
  (1, 348135)	0.4739279595416274
  (1, 256777)	0.28751585696559306
  (1, 217562)	0.40288153995289894
  (1, 145393)	0.575262969264869
  (1, 15110)	0.211037449588008
  (1, 6463)	0.30733520460524466
  (2, 400621)	0.4317732461913093
  (2, 256834)	0.2564939661498776
  (2, 183312)	0.5892069252021465
  (2, 89448)	0.36340369428387626
  (2, 34401)	0.37916255084357414
  :	:
  (319994, 123278)	0.4530341382559843
  (319995, 444934)	0.3211092817599261
  (319995, 420984)	0.22631428606830145
  (319995, 416257)	0.23816465111736276
  (319995, 3

training Ml model logistic model

logistic regression


In [34]:
model = LogisticRegression(max_iter=1000) #loading the model


In [35]:
model.fit(x_train,y_train) #training the model

LogisticRegression(max_iter=1000)

model evolvation

In [36]:
#accuracy_score
#accuracy score on training data
x_train_prediction = model.predict(x_train)  #predicting the data
training_data_accuracy = accuracy_score(x_train_prediction,y_train) #comparing the predicted value

In [37]:
print('accuracy on training data :',training_data_accuracy) #printing the accuracy

accuracy on training data : 0.81018984375


In [38]:
#accuracy score on testing data
x_test_prediction = model.predict(x_test)  #predicting the data
testing_data_accuracy = accuracy_score(x_test_prediction,y_test) #comparing the predicted value

In [39]:
print('accuracy on testing data :',testing_data_accuracy) #printing the accuracy

accuracy on testing data : 0.7780375


model accuracy=77.8%

saing the training model

In [41]:
import pickle #to save the model


In [42]:
filename = 'trained_model.sav'  #name of the file
pickle.dump(model,open(filename,'wb')) #dumping the model

using the saved model for predictoion

In [43]:
pickle_model = pickle.load(open("/content/trained_model.sav",'rb')) #loading the model


In [45]:
x_new = x_test[3]
print(y_test[3]) #actual value
pridiction = model.predict(x_new) #predicting the value
print(pridiction) #printing the value

if (pridiction[0]==0):
  print('negative')
else:
  print('positive')

0
[0]
negative
